In [ ]:
import os
import random
from pydub import AudioSegment

In [ ]:

# Function to randomly select a background sound and a segment from it
def get_random_background_segment(background_sounds, segment_length):
    # Randomly select a background sound
    bg_sound = random.choice(background_sounds)
    
    # Get a random segment of the specified length from the selected background sound
    start_ms = random.randint(0, len(bg_sound) - segment_length)
    return bg_sound[start_ms:start_ms + segment_length]

# Load all your background sound files into a list
background_sounds = [
    AudioSegment.from_file("forest_sound1.wav"),
    AudioSegment.from_file("forest_sound2.wav"),
    AudioSegment.from_file("water_sound.wav"),
    AudioSegment.from_file("wind_sound.wav"),
]


In [ ]:

# Get the list of all species class directories
species_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and not d.startswith(('forest_', 'water_', 'wind_', 'background_sounds'))]

# Calculate the threshold for a class to be considered as having "less data"
file_counts = [len(os.listdir(d)) for d in species_dirs]
threshold = sum(file_counts) // len(file_counts)

# Loop through each species class directory
for species_dir in species_dirs:
    # Get the list of all animal soundfiles in the current species class directory
    animal_sound_files = [f for f in os.listdir(species_dir) if f.endswith('.wav')]
    
    # Create a new output directory for the current species class
    output_dir = species_dir + '_output'
    os.makedirs(output_dir, exist_ok=True)
    
    # If the class has less data, perform augmentation
    if len(animal_sound_files) < threshold:
        # Calculate the number of augmentations needed
        augmentations_needed = threshold - len(animal_sound_files)
        
        for i in range(augmentations_needed):
            # Randomlyselect an animal sound file to augment
            animal_sound_file = random.choice(animal_sound_files)
            
            # Constructthe full path to the animal sound file
            full_path = os.path.join(species_dir, animal_sound_file)

            # Load the randomly selected animal sound file
            animal_sound = AudioSegment.from_file(full_path)
            
            # Determine the length of the animal sound file in milliseconds
            animal_sound_length = len(animal_sound)

            # Get a random background soundsegment of the same length as the animal sound file
            bg_segment = get_random_background_segment(background_sounds, animal_sound_length)

            # Overlay the animal sound on top of the random background segment
            combined = bg_segment.overlay(animal_sound)

            # Construct the full path tothe output file
            output_path = os.path.join(output_dir, f'combined_{i}_{animal_sound_file}')

            # Save the result in the output directory
            combined.export(output_path, format="wav")